In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
from tqdm import tqdm
import numpy as np
from copy import deepcopy

import rospy
import actionlib

from tams_pr2_guzheng.paths import RuckigPath
from tams_pr2_guzheng.utils import *
from tams_pr2_guzheng.msg import RunEpisodeAction


In [ ]:
rospy.init_node('jupyter', anonymous=True)

In [ ]:
run_episode = actionlib.SimpleActionClient('run_episode', RunEpisodeAction)
run_episode.wait_for_server()

In [ ]:
if os.path.exists('/tmp/plucks.json'):
    plucks = pd.read_json('/tmp/plucks.json')
else:
    plucks = pd.DataFrame(columns=[])

In [ ]:
p = RuckigPath.random(note= 'b3', string_position=.17)
p.dataframe.plot(x='y', y='z')

In [ ]:
result = run_params(run_episode, p, finger= 'ff')
row = row_from_result(result)
row['params'] = result.parameters
row_df = pd.DataFrame(row, columns= row.keys(), index= [0])
plucks = pd.concat((plucks, row_df), ignore_index=True)

In [ ]:
p = deepcopy(original_p)
p.keypoint_pos = [0.0, -0.01]
p.keypoint_vel = [0.00, 0.08]
p.dataframe.plot(x='y', y='z', legend=False)

In [ ]:
original_p = deepcopy(p)

In [ ]:
p = deepcopy(original_p)

In [ ]:
def add_noise(path_original):
    path = deepcopy(path_original)
    path.keypoint_pos = np.clip(np.array(path.keypoint_pos) + np.random.normal(0.0, 5e-4, 2), -1.0, 1.0).tolist()
    vel = np.array(path.keypoint_vel) + np.random.normal(0.0, 1e-3, 2)
    vel[0] = np.clip(vel[0], *sorted(path.direction * np.array([0, 0.1])))
    vel[1] = np.clip(vel[1], 0.0, 0.008)
    path.keypoint_vel = vel.tolist()
    return path if path.feasible else path_original

In [ ]:
target= 35.0
for n in tqdm(range(5)):
    p = RuckigPath.from_action_parameters(plucks.iloc[np.abs(plucks['loudness']-target).argmin()]['params'])
    add_noise(p)
    result = run_params(run_episode, p, finger= 'ff')
    row = row_from_result(result)
    print(f"loudness: {row['loudness']}")
    row['params'] = result.parameters
    row_df = pd.DataFrame(row, columns= row.keys(), index= [0])
    plucks = pd.concat((plucks, row_df), ignore_index=True)

In [ ]:
sns.swarmplot(x= plucks['loudness'])

In [ ]:
np.abs(plucks['loudness']- 20).argmin()

In [ ]:
plucks.to_json('/tmp/plucks.json')

In [ ]:
target = 25
p = RuckigPath.from_action_parameters(plucks.iloc[np.abs(plucks['loudness']-target).argmin()]['params'])